In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
import pickle
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

/Users/judebuenaseda/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [5]:
test_features = pd.read_csv('kc_house_data_test_features.csv', index_col=0)
# training_data = pd.read_csv('kc_house_data_train.csv', index_col=0)

In [8]:
test_features.columns

Index(['id', 'date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [6]:
# price = np.log(test_features['price'])

KeyError: 'price'

In [10]:
#feature engineering

#if a house has 1 or more views, it's a 1 otherwise a 0
test_features['view_bin'] = np.where(test_features['view'].values >= 1, 1,0)
test_features['bedrooms'] = np.where((test_features.bedrooms > 6),6,test_features['bedrooms'])

In [11]:
# Continuous features to scale 
#living = training_data['sqft_living']
logliving = np.log(test_features['sqft_living'])
bed = test_features['bedrooms']
bath = test_features['bathrooms']

In [12]:
# Scale the features
scaler = StandardScaler()
#scaled_living = scaler.fit_transform(training_data[['sqft_living']])
scaled_livinglog = scaler.fit_transform(np.log(test_features[['sqft_living']]))
scaled_bed = scaler.fit_transform(test_features[['bedrooms']])
scaled_bath = scaler.fit_transform(test_features[['bathrooms']])

In [13]:
# Create a dataframe with scaled features
data_fin = pd.DataFrame([])
data_fin['living'] = scaled_livinglog.flatten()
data_fin['bed'] = scaled_bed.flatten()
data_fin['bath'] = scaled_bath.flatten()

In [17]:
# Create dummies for categorical features
grade_dummies = pd.get_dummies(test_features['grade'], prefix='grade', drop_first=True)
water = test_features['waterfront']
view = test_features['view_bin']

In [20]:
#drop the grade dummies with high p-values
grade_dummies.drop(columns=['grade_4', 'grade_5', 'grade_6', 'grade_7', 'grade_8'], inplace=True)

In [22]:
# Concat with dataframe 
data_fin = pd.concat([data_fin, grade_dummies, water, view], axis=1)

In [40]:
import pickle
# Load
with open('regression_model.pickle', 'rb') as file:
    reg2 = pickle.load(file)
f_a = reg2.predict(data_fin)

In [41]:
final_answer = np.exp(f_a)

In [43]:
final_answer

array([[448402.81999707],
       [448402.81999707],
       [363235.70220071],
       ...,
       [287549.45775023],
       [380946.14607404],
       [287549.45775023]])

In [42]:
np.savetxt("J&K_Final_Answers.csv", final_answer, delimiter=",")

In [2]:
trying = pd.read_csv('J&K_Final_Answers.csv')

In [5]:
trying[0]

KeyError: 0